# I am going to build a small language model from scrath with around 15M parameters cause I am not able to find a JOB!

## Its divided into 5 major parts
### 1. Dataset
### 2. Preprocessing(Tokenization and Input Output Pairs)
### 3. Model Architecture
### 4. Training (Pipeline and Pre-Training)
### 5. Inference

# IMPLEMENTATION

1: Importing the Dataset
TinyStories is a synthetic dataset of short stories that only contain words that a typical 3 to 4-year-olds usually understand, generated by GPT-3.5 and GPT-4. We can get it from HuggingFace.

In [1]:
!pip install datasets

In [2]:
pip install -U datasets

In [3]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

2: Tokenize the Dataset
In this step, we will do the following:

(1) Tokenize the dataset into tokenIDs.

(2) Create a file called "train.bin" and "validtion.bin" where we will store the tokenIDs from the entire dataset.

(3) We make sure the tokenIDs are stored on a disk, rather than on the RAM for efficient computations.

So LLMs use a method of tokenization called BPE- Byte Pair Encoding instead of building that from scratch we will be using GPT-2's tokenizer. BPE is a better tokenization scheme that word tokenization and charchter tokenization.

The reason why we are creating a bin file is cause the dataset consists of individual stoiries those stories will be merged together with their token ids in the bin folder

In [4]:
!pip install tiktoken
import tiktoken
import os
import numpy as np
from tqdm.auto import tqdm

In [5]:
enc = tiktoken.get_encoding("gpt2")

We have stored all the token ids in a memory mapped array which is in the train.bin file cause storing it in the RAm makes it less efficient.

Each story gets convewrted into a dictonary of IDs and Lengths and we combine those toogether and save it on disk.

In [6]:
def process(example):
    ids = enc.encode_ordinary(example['text']) # encode_ordinary ignores any special tokens
    out = {'ids': ids, 'len': len(ids)}
    return out

if not os.path.exists("train.bin"):
    tokenized = ds.map(
        process,
        remove_columns=['text'],
        desc="tokenizing the splits",
        num_proc=8,
        )
    # concatenate all the ids in each dataset into one large file we can use for training
    for split, dset in tokenized.items():
        arr_len = np.sum(dset['len'], dtype=np.uint64)
        filename = f'{split}.bin'
        dtype = np.uint16 # (can do since enc.max_token_value == 50256 is < 2**16)
        arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
        total_batches = 1024

        idx = 0
        for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
            # Batch together samples for faster write
            batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
            arr_batch = np.concatenate(batch['ids'])
            # Write into mmap
            arr[idx : idx + len(arr_batch)] = arr_batch
            idx += len(arr_batch)
        arr.flush()

tokenizing the splits (num_proc=8):   0%|          | 0/2119719 [00:00<?, ? examples/s]

tokenizing the splits (num_proc=8):   0%|          | 0/21990 [00:00<?, ? examples/s]

writing train.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

writing validation.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

3. Input-Output Pairs for the dataset

In normal ML models there is a correct answer i.r. a lable so what is the correct label for LLM's?

The insput is the input text which is given and the output is to predict the next word

Context Size : The number of tokens our model looks at before predicting the next word. In opur case its 4

Then we define a input Tensor Matrix whose size is Context Size By Batch Size in our case 4X4

The output of the input tenosr matrix will be the input tensor matrix shifter by one

In [7]:
def get_batch(split):
    # We recreate np.memmap every batch to avoid a memory leak
    if split == 'train':
        data = np.memmap('train.bin', dtype=np.uint16, mode='r')
    else:
        data = np.memmap('validation.bin', dtype=np.uint16, mode='r')
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y


4. Defining the architecture

1. Layer Normalization  -1
2. Multi Head Attention
3. Dropout
4. Layer Norm -2
5. Forward NN
6. Dropout

We convert tokens into higher dimentional vectors to while we hpope to catch the semantic meanings between the words. We maintain a token embeddings matrix of the convewrted tokens of rows same a no of token ids i.e. vocabulary size by embedding dimension.
Embedding dimention u can choose whatever you want.

Thw whole matix is initialized randomly. Nad we oerform layer normalization to reduce the probability of vanishing gradient and exploding gradient.

Up untill this point we have converted workds into tokens and created a matrix also but we have no idea about how the tokens corelate to each other. This is where the attention mechanism comes in.

Through the multihead attention mechanism we calculate the attention score between one token and all the other tokens around it. With this information we will create a context vector.

The way we go from input vectors to contect vectors is through 3 matrices which are trainable named Query, Key, Value.

Firstly we multiple Token Embeddings matrix with each of the 3 marices(Dot Product). After that we multiply query with key.By multiplying these 2 we get attention score matix, it tells us the score between each of the input words.Its then converted into a attention weights matrix. Then this is multiplied with Keys whih gives us the context vector matix which is the output of the multihead attention.

There is another thing calles Causal Attention block, what is does basically it stes the values above the digonal to 0, As in each token only sees previous tokens not the upcoming tokens.

Its much more deeper than this , the matirx is normalized and we use softmax function and many other things this is just a general idea.

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
import numpy as np
from tqdm.auto import tqdm
from contextlib import nullcontext
import os

class LayerNorm(nn.Module):
    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None
    def forward(self, x):
        return F.layer_norm(x, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.flash = hasattr(F, 'scaled_dot_product_attention')
        if not self.flash:
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                       .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        if self.flash:
            y = F.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.attn_dropout.p if self.training else 0.0, is_causal=True)
        else:
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)
    def forward(self, x):
        return self.dropout(self.c_proj(self.gelu(self.c_fc(x))))

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = LayerNorm(config.n_embd, config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln2 = LayerNorm(config.n_embd, config.bias)
        self.mlp = MLP(config)
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int
    vocab_size: int
    n_layer: int
    n_head: int
    n_embd: int
    dropout: float = 0.0
    bias: bool = True

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),
            wpe=nn.Embedding(config.block_size, config.n_embd),
            drop=nn.Dropout(config.dropout),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=LayerNorm(config.n_embd, config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight  # weight tying

        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size
        pos = torch.arange(0, t, dtype=torch.long, device=device)

        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(pos)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
            return logits, loss
        else:
            logits = self.lm_head(x[:, [-1], :])
            return logits, None

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Generate tokens given a conditioning sequence.
        idx: Tensor of shape (B, T)
        """
        for _ in range(max_new_tokens):
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx



In [9]:
config = GPTConfig(
    vocab_size=50257,     # use the tokenizer's vocab size
    block_size=128,       # or whatever context size you're training with
    n_layer=6,
    n_head=6,
    n_embd=384,
    dropout=0.1,
    bias=True
)

model = GPT(config)

4.1 Loss function

After the input text (a sequence of tokens) is processed through the SLM's input layer, transformer (processor) block, and output head, the model produces a logits matrix.

Converting Logits to Probabilities (Softmax):

 Defining Target Tokens:

The Goal of the Loss Function:

The primary goal is to maximize the probability assigned by the model to the correct next token (the target token ID) for each prediction task

In [10]:
def estimate_loss(model):
    out = {}
    model.eval()
    with torch.inference_mode():
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                with ctx:
                    logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
    model.train()
    return out

Training Configurations

In [11]:
# Training Config
import torch
from contextlib import nullcontext

learning_rate = 1e-4 #more stable training
max_iters = 20000
warmup_steps = 1000 #smoother initial train, earlier 100
min_lr = 5e-4 #lower rate, earlier 5e-4
eval_iters = 500 # increased from 100
batch_size = 32 # changed from 16, better gradient estimate
block_size = 128 #changed from 64, capture longer range dependencies

gradient_accumulation_steps = 32 # reduced from 50

device =  "cuda" if torch.cuda.is_available() else "cpu"
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast


dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]

ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

torch.set_default_device(device)
torch.manual_seed(42)

In [12]:
from torch.optim.lr_scheduler import LinearLR,SequentialLR, CosineAnnealingLR

##PUT IN WEIGHT DECAY, CHANGED BETA2 to 0.95
optimizer =  torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.95), weight_decay=0.1, eps=1e-9) #weight decay for regularization

scheduler_warmup = LinearLR(optimizer, total_iters = warmup_steps) #Implement linear warmup
scheduler_decay = CosineAnnealingLR(optimizer,T_max = max_iters - warmup_steps, eta_min = min_lr) #Implement lr decay
scheduler = SequentialLR(optimizer, schedulers=[scheduler_warmup, scheduler_decay], milestones=[warmup_steps]) #Switching from warmup to decay


scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

/tmp/ipython-input-12-1497637554.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


PreTraining the SLM

This is the iterative process where the model learns and its parameters are updated. The loop typically runs for a predefined number of iterations (e.g., 20,000).

1. Get Batch (X, Y): A random batch of input (X) and target (Y) token IDs is obtained using the get_batch function.

2. Forward Pass: The input X is passed through the entire SLM architecture (input, processor, output blocks) to generate the logits matrix.

3. Calculate Loss: The cross-entropy loss is computed between the model's logits and the true targets (Y). The loss is divided by the gradient accumulation steps to prepare for accumulation.

4. Backward Pass: The calculated loss is backpropagated through the model to compute the gradients for all trainable parameters. These gradients are accumulated over the specified gradient_accumulation_steps.

5. Parameter Update: Only after the gradients have been accumulated for the set number of steps are the model's parameters updated using the AdamW optimizer.

6. Learning Rate Update: The learning rate is updated according to the defined schedule (warm-up and cosine decay) after parameters are updated.

7. Evaluate and Report Loss: Periodically (e.g., every 500 iterations), the estimate_loss function is called to calculate and print the average training and validation loss. This helps monitor the model's progress and check for overfitting.

8. Save Best Model: The model parameters that result in the best validation loss are saved to a file (best_model_params.pt) so that the trained model can be reloaded later without retraining.

In [13]:
best_val_loss = float('inf')
best_model_params_path = "best_model_params.pt"
train_loss_list, validation_loss_list = [], []

# Ensure model is on the correct device
model = model.to(device)

# In your training loop
for epoch in tqdm(range(max_iters)):
    if epoch % eval_iters == 0 and epoch != 0:
        # Ensure estimate_loss uses the correct device
        losses = estimate_loss(model)
        print(f"Epoch {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        print(f"The current learning rate: {optimizer.param_groups[0]['lr']:.5f}")
        train_loss_list += [losses['train']]
        validation_loss_list += [losses['val']]

        if losses['val'] < best_val_loss:
            best_val_loss = losses['val']
            torch.save(model.state_dict(), best_model_params_path)

    # Ensure X and y are on the correct device
    X, y = get_batch("train")
    X, y = X.to(device), y.to(device)

    with ctx:
        logits, loss = model(X, y)
        loss = loss / gradient_accumulation_steps
        scaler.scale(loss).backward()

    if ((epoch + 1) % gradient_accumulation_steps == 0) or (epoch + 1 == max_iters):
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)
    scheduler.step()

  0%|          | 0/20000 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 500: train loss 9.3686, val loss 9.3745
The current learning rate: 0.00007


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 1000: train loss 8.3970, val loss 8.4026
The current learning rate: 0.00010
Epoch 1500: train loss 7.4466, val loss 7.4457
The current learning rate: 0.00010
Epoch 2000: train loss 6.6038, val loss 6.6021
The current learning rate: 0.00010
Epoch 2500: train loss 5.9121, val loss 5.9128
The current learning rate: 0.00011
Epoch 3000: train loss 5.4165, val loss 5.4164
The current learning rate: 0.00011
Epoch 3500: train loss 5.0218, val loss 5.0218
The current learning rate: 0.00012
Epoch 4000: train loss 4.7129, val loss 4.7121
The current learning rate: 0.00012
Epoch 4500: train loss 4.4818, val loss 4.4683
The current learning rate: 0.00013
Epoch 5000: train loss 4.2627, val loss 4.2721
The current learning rate: 0.00014
Epoch 5500: train loss 4.1051, val loss 4.1054
The current learning rate: 0.00015
Epoch 6000: train loss 3.9544, val loss 3.9525
The current learning rate: 0.00016
Epoch 6500: train loss 3.8157, val loss 3.8189
The current learning rate: 0.00018
Epoch 7000: trai

Running the inference "

In [14]:
#Load the model
model = GPT(config)  # re-create the model with same config
device =  "cuda" if torch.cuda.is_available() else "cpu"
best_model_params_path = "best_model_params.pt"
model.load_state_dict(torch.load(best_model_params_path, map_location=torch.device(device))) # load best model states


<All keys matched successfully>

In [15]:
sentence = "Once upon a time there was a pumpkin."
context = (torch.tensor(enc.encode_ordinary(sentence)).unsqueeze(dim = 0))
y = model.generate(context, 200)
print(enc.decode(y.squeeze().tolist()))

Once upon a time there was a pumpkin. It was very special. The pumpkin were very happy. It liked it coming to a farm. 

One day, a mother looked out the beyond the water and saw the eggs had 2 laser. She picked it up and mommy went inside to receive the volcano. 

Inside the island, the pumpkin found a board. Molly took it through the map. She found it under it and felt so happy! 

The mother Dad saw Alice that the model was getting gone. He was so relaxed and cozy made her feel warm and full. Little Thursday workerAlright and child return the eggs. Ann was very happy and younger with the Baby, so she ran to where Mommy and daddy was so excited to see the seeds back down the sea. Everyone was very proud of her!Once upon a time, there was a man. It was a very special kind of him. He liked to wear his pretty clothes and feet. He always made sure to cool things forever so he


In [16]:
sentence = "A little girl went to the woods"
context = (torch.tensor(enc.encode_ordinary(sentence)).unsqueeze(dim = 0))
y = model.generate(context, 200)
print(enc.decode(y.squeeze().tolist()))

A little girl went to the woods. She was looking at the animals and their kind old boat. She saw a jadding on a singer soaring around her. She was so happy and soon she could not want to balloon any problems.

She asked her mom, "Where do we do that the dolphin will feel rough. I don't know that you need a little, I think I've got inside. I'll have a thief to your plane get off."

Her mom found herself in a towel and Lea showed Lily the plane to her mom. The bus was old and came inside. They buses were happy the passengers and colorful. They said, "Goodbye, mom and the van!" Their mom and the plane rise down, and put them in one bathtub and said they had a great adventure!Once upon a time, there was a chubby little boy named Jack. He loved to ride around his shadow.

One day, Jack found a trick on the perfect spot with his fourth party
